<a href="https://colab.research.google.com/github/k00lk0der1/synthpoppp/blob/main/Pipeline_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install sdv
!pip3 install git+https://github.com/k00lk0der1/synthpoppp/

!wget https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km/2020/IND/ind_pd_2020_1km_ASCII_XYZ.zip
!unzip ind_pd_2020_1km_ASCII_XYZ.zip

In [ ]:
from synthpoppp.base_population import IPU
from synthpoppp.helper_constants import POSSIBLE_JOB_LABELS
from synthpoppp.geographical_distributions import HLatHlongAgeAddition, JobsPlacesAddition, PopulationDensitySampler
import pandas as pd
import numpy as np

import gc
import os

In [ ]:
###Configuration

region_name = "haryana/ambala"
state_id = 6 ####### select state number and name appropriately
district_ids = [] #select district number

district_wise_filtering = False

population_density_filename = "ind_pd_2020_1km_ASCII_XYZ.csv"

working_dir = "/content/drive/MyDrive/data/synthetic/"

ihds_individuals_filename = working_dir+"36151-0001-Data.tsv"
ihds_household_filename = working_dir+"36151-0002-Data.tsv"

householdh_marginal_filename = working_dir+region_name+"/household_marg.csv"
individuals_marginal_filename = working_dir+region_name+"/person_marg.csv"

admin_units_geojson_filename = working_dir+region_name+"/admin_units.geojson"
admin_units_population_filename = working_dir+region_name+"/admin_unit_wise_pop.csv"

if(district_wise_filtering):
    synthetic_population_output_filename = working_dir+region_name+"/synthetic.csv"
else:
    synthetic_population_output_filename = working_dir+region_name+"/synthetic_state_wise.csv"

In [ ]:
ihds_individuals_data = pd.read_csv(ihds_individuals_filename, sep='\t')

filtered_ihds_individuals_data = ihds_individuals_data.loc[ihds_individuals_data.STATEID==state_id]

ihds_households_data = pd.read_csv(ihds_household_filename, sep='\t')

filtered_ihds_households_data = ihds_households_data.loc[ihds_households_data.STATEID==state_id] 

if(district_wise_filtering):
    filtered_ihds_individuals_data = filtered_ihds_individuals_data[filtered_ihds_individuals_data['DISTID'].apply(lambda x : x in district_ids)]   
    filtered_ihds_households_data = filtered_ihds_households_data[filtered_ihds_households_data['DISTID'].apply(lambda x : x in district_ids)]

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,6,9,10,11,12,13,14,15,16,17,18,22,27,29,33,34,39,44,48,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,74,75,76,78,79,80,88,91,105,150,151,152,154,158,159,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,194,196,197,198,199,200,201,202,203,204,205,217,218,219,220,221,222,223,224,225,226,227,228,229,230,234,235,236,237,240,241,243,245,247,248,249,250,275,279,283,287,288,289,290,291,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,320,322,334) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (14,15,16,17,18,19,20,21,24,25,26,28,31,35,36,113,114,115,116,117,118,119,120,121,122,126,186,187,188,189,190,191,192,193,194,212,213,214

In [ ]:
filtered_ihds_households_data

,SURVEY,STATEID,DISTID,PSUID,HHID,HHSPLITID,IDPSU,IDHH,WT,INDWT,...,NWKANY5,INCNONAG,INCAGLAB,INCSALARY,INCNREGA,INCNONNREGA,NNR,HHEDUC,HHEDUCM,HHEDUCF
4451,2,6,1,1,21,1,60101,601010211,7223.443234,43340.659403,...,1,22500,0,0,0,22500,0,0,0,0
4452,2,6,1,1,31,1,60101,601010311,7223.443234,28893.772935,...,2,0,0,0,0,0,0,0,0,0
4453,2,6,1,1,32,1,60101,601010321,7223.443234,28893.772935,...,2,0,0,0,0,0,0,8,8,5
4454,2,6,1,1,41,1,60101,601010411,7223.443234,86681.318805,...,4,0,0,48000,0,0,0,12,12,12
4455,2,6,1,1,51,1,60101,601010511,7223.443234,57787.545870,...,1,0,0,120000,0,0,0,6,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6252,2,6,19,9,110,1,61909,619091101,4407.666667,8815.333333,...,1,0,0,72000,0,0,0,10,9,10
6253,2,6,19,9,110,2,61909,619091102,4407.666667,13223.000000,...,1,36000,0,0,0,36000,0,12,12,12
6254,2,6,19,9,130,1,61909,619091301,4407.666667,22038.333333,...,3,0,0,138000,0,0,0,15,15,8
6255,2,6,19,9,140,1,61909,619091401,4407.666667,35261.333333,...,3,0,0,304800,0,0,0,16,15,16


In [ ]:
filtered_ihds_individuals_data

,SURVEY,STATEID,DISTID,PSUID,HHID,HHSPLITID,PERSONID,IDPSU,IDHH,IDPERSON,...,MG8,MG9NM,MG10,MG11,MGYEAR5,NMIG5,MGMONTHS5,MGYEAR1,NMIG1,MGMONTHS1
21882,2,6,1,1,21,1,1,60101,601010211,60101021101,...,,,,,0,,,0,,
21883,2,6,1,1,21,1,2,60101,601010211,60101021102,...,,,,,0,,,0,,
21884,2,6,1,1,21,1,3,60101,601010211,60101021103,...,,,,,0,,,0,,
21885,2,6,1,1,21,1,4,60101,601010211,60101021104,...,,,,,0,,,0,,
21886,2,6,1,1,21,1,5,60101,601010211,60101021105,...,,,,,0,,,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31445,2,6,19,9,140,1,8,61909,619091401,61909140108,...,,,,,0,,,0,,
31446,2,6,19,9,150,1,1,61909,619091501,61909150101,...,,,,,0,,,0,,
31447,2,6,19,9,150,1,2,61909,619091501,61909150102,...,,,,,0,,,0,,
31448,2,6,19,9,150,1,3,61909,619091501,61909150103,...,,,,,0,,,0,,


In [ ]:
ipu_object = IPU()
synthetic_households, synthetic_individuals, synthetic_stats = ipu_object.generate_data(filtered_ihds_individuals_data, filtered_ihds_households_data, householdh_marginal_filename, individuals_marginal_filename)

display(synthetic_individuals)
display(synthetic_households)

hlat_hlong_age_object = HLatHlongAgeAddition(admin_units_geojson_filename, admin_units_population_filename, population_density_filename)
new_synthetic_population = hlat_hlong_age_object.perform_transforms(synthetic_individuals, synthetic_households)

display(new_synthetic_population)

jobs_places_object = JobsPlacesAddition(POSSIBLE_JOB_LABELS, admin_units_geojson_filename, 10000, 10000, population_density_filename, 'zipf', 'default', 'default')
new_synthetic_population = jobs_places_object.perform_transforms(new_synthetic_population) 

,district,serialno,mem_id,SexLabel,Age,religion,caste
21882,1,601010211,1,Male,35to39,Hindu,Other
21883,1,601010211,2,Female,30to34,Hindu,Other
21884,1,601010211,3,Female,10to14,Hindu,Other
21885,1,601010211,4,Female,5to9,Hindu,Other
21886,1,601010211,5,Male,5to9,Hindu,Other
...,...,...,...,...,...,...,...
31445,19,619091401,8,Female,0to4,Hindu,Other
31446,19,619091501,1,Male,70to74,Hindu,Other
31447,19,619091501,2,Female,60to64,Hindu,Other
31448,19,619091501,3,Male,25to29,Hindu,Other


,district,serialno,hhsize
4451,1,601010211,hhsize_6
4452,1,601010311,hhsize_4
4453,1,601010321,hhsize_4
4454,1,601010411,hhsize_1114
4455,1,601010511,hhsize_710
...,...,...,...
6252,19,619091101,hhsize_2
6253,19,619091102,hhsize_3
6254,19,619091301,hhsize_5
6255,19,619091401,hhsize_710


district             distid   hhsize                                      \
         Unnamed: 1_level_1 hhsize_1 hhsize_2 hhsize_3 hhsize_4 hhsize_5   
ambala                   70   6176.0  14452.0  25456.0  55634.0  48970.0   

district                                             
         hhsize_6 hhsize_710 hhsize_1114 hhsize_15p  
ambala    31558.0    34675.0      4508.0     1085.0

district distid  total_pop  SexLabel                Age                     \
         distid  total_pop      Male    Female     0to4     5to9    10to14   
ambala       70  1128350.0  598703.0  529647.0  89510.0  94872.0  100432.0   

district                                ...  religion                    \
            15to19    20to24    25to29  ...     Hindu  Muslim Christian   
ambala    110758.0  116232.0  106055.0  ...  137047.0  7693.0    1094.0   

district                                   caste             
             Sikh Buddhist   Jain  Other      SC ST   Other  
ambala    21888.0     50.0  139.0  162.0  296246  0  832104  

[1 rows x 31 columns]

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/usr/local/lib/python3.8/dist-packages/synthpop/ipu/ipu.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  adj = constraint / float((column * weights).sum())


...

100%|██████████| 2123960/2123960 [01:28<00:00, 24020.95it/s]


,district,serialno,mem_id,SexLabel,Age,religion,caste,sample_geog,cat_id,geog,household_id,H_Lat,H_Lon,AdminUnitName,AdminUnitLatitude,AdminUnitLongitude
0,1,601010211,1,Male,36,Hindu,Other,1,158,ambala,151385,30.514185,77.180377,ambala,30.375201,76.782122
1,1,601010211,2,Female,31,Hindu,Other,1,494,ambala,151385,30.514185,77.180377,ambala,30.375201,76.782122
2,1,601010211,3,Female,10,Hindu,Other,1,410,ambala,151385,30.514185,77.180377,ambala,30.375201,76.782122
3,1,601010211,4,Female,9,Hindu,Other,1,389,ambala,151385,30.514185,77.180377,ambala,30.375201,76.782122
4,1,601010211,5,Male,9,Hindu,Other,1,32,ambala,151385,30.514185,77.180377,ambala,30.375201,76.782122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064011,18,618090501,1,Male,44,Jain,Other,1,194,ambala,150518,30.422317,76.986346,ambala,30.375201,76.782122
1064012,18,618090501,4,Male,11,Jain,Other,1,68,ambala,150518,30.422317,76.986346,ambala,30.375201,76.782122
1064013,18,618090501,2,Female,41,Jain,Other,1,551,ambala,150518,30.422317,76.986346,ambala,30.375201,76.782122
1064014,18,618090501,3,Female,16,Jain,Other,1,446,ambala,150518,30.422317,76.986346,ambala,30.375201,76.782122


100%|██████████| 4010402/4010402 [01:50<00:00, 36152.94it/s]


10001


100%|██████████| 100000/100000 [00:04<00:00, 24233.41it/s]
/usr/local/lib/python3.8/dist-packages/synthpoppp/geographical_distributions.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['JobType'] = np.random.choice(self.job_type_list, size=(adults.shape[0],))
/usr/local/lib/python3.8/dist-packages/synthpoppp/geographical_distributions.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['WorksAtSameCategory'] = np.random.uniform(size=(adults.shape[0],))>0.05
100%|██████████| 186/186 [00:

In [ ]:
new_synthetic_population.sample(n=10000).reset_index(drop=True).to_csv(synthetic_population_output_filename+".sample", index=False)
new_synthetic_population.to_csv(synthetic_population_output_filename, index=False)